In [16]:

import urllib.request
import pandas as pd
import numpy as np
import pickle

import datetime

api_key = "API_KEY"
start_date = str(datetime.date.today() - datetime.timedelta(days = 5))
end_date = str(datetime.date.today())



In [17]:
start_date

'2023-10-06'

In [18]:
start_date
end_date

'2023-10-11'

In [19]:
url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Central%20Park%20New%20York/{start_date}/{end_date}?unitGroup=us&maxDistance=1000&include=hours&key={api_key}&elevationDifference=1&contentType=csv'
destination = 'Data/infer_temp.csv'

urllib.request.urlretrieve(url, destination)

('Data/infer_temp.csv', <http.client.HTTPMessage at 0x115989790>)

In [20]:
df = pd.read_csv(destination)
# df.head()

In [34]:
file = open("models/xgb_reg_final.pkl",'rb')
model = pickle.load(file)
file.close()

In [43]:
def give_max_pred(model, df):

    df = df[['datetime', 'temp', 'precip', 'humidity', 'windspeed', 'sealevelpressure', 'winddir']]

    df['datetime'] = pd.to_datetime(df['datetime'])

    df['date'] = df['datetime'].dt.date

    day_df_max = df.groupby('date')[['temp', 'precip', 'humidity', 'windspeed']].agg('max')
    day_df_min = df.groupby('date')[['temp', 'precip', 'humidity', 'windspeed']].agg('min')

    day_df_max['max_precip_prev1d'] = day_df_max['precip'].rolling(1).max()
    day_df_max['max_precip_prev3d'] = day_df_max['precip'].rolling(3).max()
    
    day_df_max['max_temp_prev1d'] = day_df_max['temp'].rolling(1).max()
    day_df_max['max_temp_prev3d'] = day_df_max['temp'].rolling(3).max()

    day_df_max['min_precip_prev1d'] = day_df_min['precip'].rolling(1).min()
    day_df_max['min_precip_prev3d'] = day_df_min['precip'].rolling(3).min()
    
    day_df_max['min_temp_prev1d'] = day_df_min['temp'].rolling(1).min()
    day_df_max['min_temp_prev3d'] = day_df_min['temp'].rolling(3).min()

    day_df_max.drop(['temp', 'precip', 'humidity', 'windspeed'], axis = 1, inplace = True)


    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    df['hour'] = df['datetime'].dt.hour

    df = pd.merge(df, day_df_max, on = 'date', how = 'left')

    for i in range(1,24):
        df[f'temp_lag_{i}'] = df['temp'].shift(i)

    df = df.iloc[72:]

    df.fillna(-1, inplace = True)

    curr_hr = datetime.datetime.now().hour
    curr_date = str(datetime.date.today())


    df = df[~((df['date'] == pd.to_datetime(curr_date).date()) & (df['hour'] > curr_hr))]

    df.drop(['datetime'], axis = 1, inplace = True)

    X_test = df.drop(['date'], axis = 1)

    y_pred = model.predict(X_test)

    return y_pred[0]



In [44]:
give_max_pred(model, df)

/var/folders/x5/1285xgcd3vl1lvxv_nmsdqtc0000gn/T/ipykernel_973/881859357.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = pd.to_datetime(df['datetime'])
/var/folders/x5/1285xgcd3vl1lvxv_nmsdqtc0000gn/T/ipykernel_973/881859357.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['datetime'].dt.date
/var/folders/x5/1285xgcd3vl1lvxv_nmsdqtc0000gn/T/ipykernel_973/881859357.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

57.813698